 # 隐马尔科夫模型和它的问题与算法
 
 ## 不隐的马尔科夫模型
 
 马尔科夫性就是说只有前一个状态会影响后面的状态，更前面的则不会。即
 
 $$
 P(X_t = x_t \mid X_{t-1} = x_{t-1} , X_{t-2} = x_{t-2}, \dots) = P(X_t = x_t \mid X_{t-1} = x_{t-1})
 $$

从推导的角度看，这个性质可以用来分解联合概率（由于很多直观意义不太明显的项经常会还原到完全的联合分布再“边缘”回去，所以这很重要）

$$
\begin{align}
P(X_t = x_t,X_{t-1} = x_{t-1},\dots,X_1 = x_1) 
&= P(X_1 = x_1) P(X_2 = x_2 \mid X_1 = x_1) P(X_3 = x_3 \mid X_1 = x_1, X_2 = x_2) \cdots P(X_n = x_n \mid X_{n-1} = x_{n-1},X_{n-2} = x_{n-2},\cdots,X_1 = x_1) \\
&= P(X_1 = x_1) P(X_2 = x_2 \mid X_1 = x_1) P(X_3 = x_3 \mid X_2 = x_2) \cdots P(X_n = x_n \mid X_{n-1} = x_{n-1})
\end{align}
$$

于是我们给出初始概率与状态转移矩阵去建立马尔科夫链之类的不隐的马尔科夫模型。这些随机模型可以表示某种状态变化的规律，可以算VaR之类的东西，
虽然表达能力不怎么样，但好歹可以再各状态中切换。

比如在东方，如果我们只考虑可能的9种移动方式，则可以设一个在9种移动方式中转移的马尔科夫链，可以想象该链应该有很高的自返概率。
虽然我们都知道自机的移动显然不是“随机”的，但在没有游戏内容（模型）的背景下，我们也只能把它看成随机的。那么在所有可能的模型里，
我们可以找到看起来最符合观测轨道的那个模型——极大似然估计。

## 隐马尔科夫模型

隐马尔科夫模型（Hidden Makov Model,HMM）里包含一个马尔科夫模型，不过这里的状态不再是可观测的，而是隐状态，
我们观测到的是隐状态的emission,由于不同的隐状态可以发射相同的观测，所以我们无法通过观测直接确定隐状态。

隐马尔科夫模型一个主要用途就是想对那些可以利用状态序列信息改善预测的滤波问题建模，如果只有发射矩阵的话，
对一个观测我们想找到最有可能的隐状态是平凡的。我们只要遍历隐状态取那个对此观测概率最高的隐状态即可。
对一个序列来说固然可以在每个观测上都用一次这个方法来推断隐状态，但这是个拙劣的方法，
比如假如我们知道前一个状态很容易得出下一个状态取B有99%概率，取C有0.1%概率，而B产生对应观测的概率是30%，C是31%，则拙劣预测将给出C，
而从序列的角度看则似乎应该是B，序列模型通过考虑给定观测条件下，隐状态概率最大者来充分利用序列上的信息。

$$
\mathrm{argmax}_{x_1,\dots,x_n} = P(X_1 = x_1,\dots,X_n = X_n, Y_1 = y_1 \dots Y_n = y_n \mid Y_1 = y_1 \dots Y_n = y_n) = P(X_1 = x_1,\dots,X_n = X_n \mid Y_1 = y_1 \dots Y_n = y_n)
$$

考虑如果没有观测，隐状态序列概率最大者是有益的，容易想到那个序列将是每步从初始概率向量或转移矩阵中选出对应的最高概率状态转移的状态序列。
这是平凡的，但给定观测的条件后就不平凡了。

没有观测下预测最可能轨道其实可以退化回不隐的马尔科夫链问题，我们还可以增加一些复杂度，比如我们只知道中间几步的状态下估计最有可能轨道。
容易想到若知道的是最后的状态，我们可以利用类似动态规划的算法求解出最有可能轨道（每步求出在倒数第几步的最优概率），这就是后向算法。
知道的是中间一步的话，则更前面的部分可以用后向算法，后面的部分可以用平凡预测的方法（前向算法）。如果知道好几个点的状态的话，
为了预测那些不知道状态的点的状态，由于马尔科夫性，只需要考虑其最邻近的左边和右边的点，但我一下没想到什么明显的简化算法，
只能还原成最大化条件概率为最大化联合概率，然后穷举最优序列了。

类似于上面的在条件上找最优状态的问题，HMM的Filtering，Smoothing，Most likely explanation问题分别在给定观测
（但不是隐状态的观测，是它的发射）序列下求最优可能的最后隐状态，中间隐状态与整个隐状态序列。其中最难的最后一个问题可以用Veterbi算法解决，
用的是类似前面的前向预测的动态规划方法，但是加入了融合观测的方法，但还是比较平凡。

这三种问题都是已知参数（如果不把隐状态看成参数的话）与观测推断隐状态的问题，都有快速的精确算法求解。给定参数求序列概率
（即边缘掉隐状态）之类的更平凡的问题就不说了。那么假如参数未知，我们如何估计参数呢？

隐马尔科夫模型有三个参数$\theta = (A,B,\pi)$，其中$N \times N$矩阵，为$N$个隐状态的转移概率矩阵。$B$是$N \times K$矩阵，
每行代表对应状态产生$K$个离散观测取值的概率。$\pi$是$N$维向量，为初始状态向量。

这个优化的平凡版本当然是穷举所有可能取值（格搜索），或者把所有参数塞在一个向量里加上约束做梯度下降优化——优化目标是在该参数下，
产生出观测序列的边缘概率（即我们提到但跳过的那个“平凡”问题，虽然平凡，但也不是加减乘除一下的计算量）最大化。

若把参数看成随机变量，则可以利用MCMC得出参数的后验分布，再求MAP或期望之类的。
但此处的通过边缘化掉隐状态直接建立参数和观测的关系好像不是那么简单（虽然计算一个特例是简单地）。

在知道观测序列以外的信息情况下，问题可以简单一些，比如我们实际知道隐状态（那“隐”马尔科夫模型的“隐”体现在哪。。）序列和观测序列下，
我们可以用不隐马尔科夫链的方法估计转移矩阵$A$，再分别平凡的用频率估计$B$。$\pi$不好估计，因为我们实际上只看到该参数的一次应用。

## EM算法

EM算法是估计带隐变量的随机模型的参数的方法。大致上说隐变量和可观测变量在这种模型中都是随机的都被确定模型生成一次，但隐变量观测不到。
一种估计参数的方法是边缘化掉隐变量，然后直接估计。这常常会面临类似贝叶斯统计的积分的困难。另一种思路就是利用EM算法。

$$
Q(\theta \mid \theta^{(t)}) = \mathrm{E}_{Z \mid X,\theta^{(t)}} [\log L(\theta;X,Z)] \\
\theta^{(t+1)} = \mathrm{argmax}_\theta Q(\theta \mid \theta^{(t)})
$$

其中$\log L(\theta;X,Z)$是对数似然函数$\log L(\theta;X,Z) = \log P(X,Z \mid \theta)$，$Z$是隐变量。$X$是可观测变量。

我们应当注意到E步（Expectation）算的不是隐变量$Z$的期望，而是对数似然函数，
虽然这个函数的计算很可能会用上隐变量$Z$在$\theta$下给定$X$的条件期望，
但本身并不是计算这个条件期望。M步(maximization)则是最大化那个对数似然函数，取可以取得最大对数似然的参数作为这次迭代的结果。

对数似然函数中$\theta$是未知参数，$X$给定，$Z$用$Z \mid X,\theta_{(t)}$上的条件分布给条件期望掉了。

### k-means与EM的类似性与硬软分配

我们来考虑$R^1$上加上概率解释的k-means聚类，设各个类是$N(\theta_i,1^2)$分布的。对N个对照变量有N个隐的隶属变量决定该对照变量服从哪个分布。

显然给定$(\theta,X,X)$的一组确定值，$\log L(\theta;X,Z)$返回一个确定标量。我们可以显示表示

$$
L(\theta;X,Z) = \prod_{i=1}^N \frac{1}{\sqrt{2\pi}}e^{-\frac{(x_i-\theta_{z_i})}{2}}
$$

$x_i$已知，为了计算对数似然对$Z \mid X,\theta^{(t)}$的条件期望以消掉这个随机变量。我们需要知道这个分布$P(Z_i = j \mid X,\theta^{(t)})$，
在k-means里这就是个离散分布，可以用个矩阵表示，我们先不管这个条件分布怎么算出来的。假设我们已经有这个分布矩阵了。于是E步可以展开了一个和式

$$
\mathrm{E}_{Z \mid X,\theta^{(t)}} [\log L(\theta;X,Z)] = \sum_{k_1}^M \dots \sum_{k_N}^{M} P(Z_1 = k_1 \dots Z_N = k_N \mid X, \theta^{(t)}) [\log L(\theta;X,(k_1,\dots,k_N))] 
$$

然后M(maximazation)步将取$\theta$来使得上式最大化作为本轮对$\theta$的迭代估计结果$\theta^{(t)}$。$Z$本身的条件期望呢?我们注意到如果替换
$Z$直接换为条件期望而不是把他看成随机变量，在外部对它求条件期望的话，解的是一个类似但不相同的问题：

$$
Q'(\theta \mid \theta^{(t)}) =  \log L(\theta;X,\mathrm{E}(Z \mid X,\theta^{(t)})) \\
\theta^{'(t+1)} = \mathrm{argmax}_\theta Q'(\theta \mid \theta^{(t)})
$$

看上去这是两个不同的问题，除非条件期望算子可以像连续算子一样“移进去”。显然一般这是不对的，如:

$$
X \sim U(-1,1) \\
f(x) = x^2 \\
E(f(X)) \neq f(E(x)) = f(0) = 0
$$

标准的k-means算法用的是看上去更好算的后者算法，凭感觉这两个算法在k-means的特例应该给出类似结果甚至是等价的。

因而，这个被称为“硬分配”版的EM算法，原版的EM算法称为软分配的。

当然，至此我们仍然每说那个隐变量的条件分布怎么来的，虽然如果用后面的算法可能可以绕过条件分布直接求出条件期望
（或者任何看上去合理的用于取代$Z$的数）。

k-means里的条件分布很好求，就是直接使用贝叶斯公式：

$$
P(Z_i = j \mid X_i = x_i, \theta) = \frac{P(X_i = x_i \mid Z_i = j , \theta) P(Z_i = j)}{P(X_i = x_i)} 
= \frac{P(X_i = x_i \mid Z_i = j ,\theta) P(Z_i = j)}{\sum_{j=1}^M P(X_i = x_i \mid Z_i = j , \theta) P(Z_i = j)}
$$

其中$P(Z_i = j)$是前一轮迭代结果或者是“均匀的”（这时可以直接消掉此项）。

## Baum–Welch算法

Baum-Welch算法是EM算法用来估计HMM那三个参数的特例，我们开始的目标仍是找到$\theta$，最大化
$p(\mathbf{X} \mid \theta) = \sum_\mathbf{Z} p(\mathbf{X}, \mathbf{Z} \mid \theta)$。
直接试图利用该等式左边最大化将面临类似上面提到的计算边缘积分的困难，当然此处问题在于要加的$\mathbf{Z}$太多了，若隐状态有$M$个，
序列有$N$长（当然即将处理的实际问题还有不只一条序列。。虽然此处是只有一条）。则我们的和式将拥有$M^N$个项。虽然在极小的情况我们可以利用分解

$$
p(\mathbf{X} \mid \theta) =
\sum_{\mathbf{Z}} p(\mathbf{X}, \mathbf{Z} \mid \theta) = 
\sum_{z_1,\dots,z_N \in 1,\dots,M }p(X_1 = x_1,\dots, X_N = x_N  ,Z_1 = z_1,\dots,Z_N = z_N \mid \theta) = \\
\sum_{\mathbf{Z}} p(Z_1 = z_1)p(X_1 = x_1 \mid Z_1 = z_1)p(Z_2 = z_2 \mid Z_1 = z_1, X_1 = x_1) p(X_2 = x_2 \mid Z_2 = z_2,Z_1 = z_1,X_1 = x_1)p(Z_3 = z_3 \mid \dots) \dots
$$

利用
$$
p(Z_{t} = z_t \mid Z_{t-1} = z_{t-1}) = p(Z_{t} = z_t \mid Z_{t-1} = z_{t-1} ,anything...) \\
p(X_{t} = x_t \mid Z_{t} = z_t) = p(X_{t} = x_t \mid Z_{t} = z_t, anything...)
$$

这两个“充分性”。上式可化简为

$$
p(\mathbf{X} \mid \theta) =
\sum_{\mathbf{Z}} p(\mathbf{X}, \mathbf{Z} \mid \theta) = 
\sum_{\mathbf{Z}} p(Z_1 = z_1)p(X_1 = x_1 \mid Z_1 = z_1)p(Z_2 = z_2 \mid Z_1 = z_1) p(X_2 = x_2 \mid Z_2 = z_2) \dots p(Z_N = z_N \mid Z_{N-1} = z_{N-1})p(X_N = x_N \mid Z_N = z_N)
$$

用前面的$\theta=(A,B,\pi)$的记号，上式可以记为：

$$
p(\mathbf{X} \mid \theta) =
\sum_{\mathbf{Z}} p(\mathbf{X}, \mathbf{Z} \mid \theta) = 
\pi_{z_1} B_{z_1,x_1} A_{z_1,z_2} B_{z_2,x_2}\dots A_{z_{N-1},z_N} B_{z_N,x_N}
$$

单独拿出$p(\mathbf{X},\mathbf{Z} \mid \theta)$以此得到对数似然函数为

$$
\log L(\theta,\mathbf{X},\mathbf{Z}) = \log p(\mathbf{X},\mathbf{Z} \mid \theta) = 
\log \pi_{z_1} + \sum_{i=2}^{N} \log A_{z_{i-1},z_{i}} + \sum_{i=1}^N \log B_{z_i,x_i}
$$

固然在某些特别简单的情况我们可以直接拿这个函数优化，但是还是宁可选择EM算法

$$
Q(\theta \mid \theta^{(t)}) = \mathrm{E}_{Z \mid X,\theta^{(t)}} [\log L(\theta;X,Z)] =
\sum_{\mathbf{Z}} p(\mathbf{Z} \mid \mathbf{X},\theta^{(t)}) \log p(\mathbf{X},\mathbf{Z} \mid \theta)
$$

当然此$Q$函数还是要遍历所有可能的$\mathbf{Z}$的取值，有$N^M$个项求和。然而可以证明此函数可以表示为

$$
Q(\theta,\theta^{(t)}) = \sum_i^{M} \gamma(z_{1i}) \log \pi_i + \sum_{i=2}^N \sum_{j=1}^M\sum_{k=1}^M \xi_t(z_{i-1,j},z_{n,k}) 
\log A_{j,k} + \sum_{i=1}^N \sum_{j=1}^M \gamma (z_{nk}) \log B_{j,i}
$$

其中
$$
\gamma(z_{i,j}) = \mathbf{E}(z_{i,j})  =\sum_{\mathbf{z}} p(\mathbf{Z} = \mathbf{z} \mid \mathbf{X}, \theta_{(t)}) z_{ij} \\
\xi(z_{i-1,j}z_{i,k}) = \mathbf{E}(z_{i-1,j}z_{i,k}) =  \sum_{\mathbf{z}} p(\mathbf{Z} = \mathbf{z} \mid \mathbf{X}, \theta_{(t)}) z_{i-1,j} z_{i,k}
$$

其中$z_{i,j}$指那个$\mathbf{z}$中第$i$轮的隐状态是否是第$j$个。本身是个随机变量。可以像上面用边缘分布求其期望。

可以看出原本指数复杂变成了三次方的复杂度，在知道$\gamma(z_{i,j})$和$\xi(z_{i-1,j}z_{i,k})$的情况下。
但依照定义这两个东西复杂度还是指数级的，这只不过是转移了复杂度而已？所幸我们指出存在高效的正向反向算法可以快速求出它。

另一方面，容易看出若计算出了$\gamma(z_{i,j})$则我们也知道了$i$时刻各个隐状态$j$所处的概率，从而可以从中选择最高者作为Most likely explanation
问题的解。也许令人困惑的是，在Q函数里出现的这个项只与$\theta^{(t)}$有关而与$\theta$无关
（与$\theta$有关的是Q函数里$\pi,A,B$那些项。），从而我们可以单独计算出这个值。这个计算是相对平凡的，唯一的难点在于如何吸收后验信息。

## 一般的概率网络

HMM模型只不过是一般概率网络的特例，一般概率网络只不过是组织一堆随机变量或分布函数的方法罢了。时序是自然把随机变量划分的方法，
再加上马尔科夫假设（即t时刻类里的随机变量，给定t-1时刻和t时刻其他随机变量的信息后，就对t-n n>2的信息“充分”了），
从一个一般的联合分布函数出发，这帮助我们分解了它，有利于看出其本质的性质。

我们可以把概率网络中的量分为三种，参数，隐变量与可观察变量。指出参数是因为有时参数也被看成随机变量。

### 边缘估计

边缘估计框架中，我们将联合分布函数中的隐变量边缘化掉，直接建立参数与可观察变量的关系，代入样本后可最大化参数的对应似然度进行估计，
这就转化成了个纯粹的最优化问题，由于边缘化涉及复杂积分或数量级爆炸的求和，所以该方法一般不实用。

### EM算法估计

EM算法迭代的优化参数估计，由随机的参数初始化开始。每步进行：

1.Expectation步，即“计算出”隐变量在上一轮迭代的参数和观测值下的条件分布，然后“计算出”联合分布函数对待定参数的似然函数
（其中可观测变量被样本代入，隐变量被用那个条件分布条件期望掉了）。说是计算，其实一般是把第二步中一些重复计算的东西先计算了缓存着。

2.Maximization步，上面又得到了一个待定参数映到似然度的函数，这就又转化为了一个最优化问题了，尽管这仅仅是迭代的一步。

EM算法利用条件期望消掉隐变量与边缘估计利用边缘化消掉隐变量有什么区别呢？设联合似然（密度，概率）函数为

$$
f(x_1,\dots,x_n,z_1,\dots,z_m \mid p_1,\dots p_k)
$$

其中$n,m$等就是表示可观测变量与隐变量的个数，至于是不是可以按时间分解为$n = t \times q, m = t \times w$之类的另说。这里不care这个。

边缘化框架求解

$$
g(x_1,\dots,x_n \mid p_1,\dots,p_k) = 
\int_{-\infty}^{\infty} \dots \int_{-\infty}^{\infty} 
f(x_1,\dots,x_n,z_1,\dots,z_m \mid p_1,\dots p_k) 
\mathrm{d}z_1 \dots \mathrm{d}z_m
$$

而EM算法框架求解：

$$
h(x_1,\dots,x_n \mid p_1,\dots,p_k) = 
\int_{-\infty}^{\infty} \dots \int_{-\infty}^{\infty} 
\log f(x_1,\dots,x_n,z_1,\dots,z_m \mid p_1,\dots p_k)
f(z_1,\dots,z_m \mid x_1,\dots,x_n,\bar{p}_1,\dots \bar{p}_k)
\mathrm{d}z_1 \dots \mathrm{d}z_m
$$

$$
f(z_1,\dots,z_m \mid x_1,\dots,x_n,\bar{p}_1,\dots \bar{p}_k) = 
\frac{
f(x_1,\dots,x_n,z_1,\dots,z_m \mid \bar{p}_1,\dots \bar{p}_k)
}{
f(x_1,\dots,x_n \mid \bar{p}_1,\dots,\bar{p}_k)
}=
\frac{
f(x_1,\dots,x_n,z_1,\dots,z_m \mid \bar{p}_1,\dots \bar{p}_k)
}{
\int_{-\infty}^{\infty} \dots \int_{-\infty}^{\infty} 
f(x_1,\dots,x_n,z_1,\dots,z_m \mid \bar{p}_1,\dots \bar{p}_k) 
\mathrm{d}z_1 \dots \mathrm{d}z_m
}
$$

当然就这么看并没有看出EM算法框架比边缘估计高到哪里去了，实际上我们发现边缘积分干脆就是EM算法计算的一部分？这不变得更麻烦了吗？
而且wikipedia上那个混合高斯分布的其实也是解出了那个积分，结果EM单纯就变成了一种特定的迭代法。。
看起来并没有比直接在边缘似然函数用数值迭代法高到哪里去（EM算法本身也不保证最优解）。

你说HMM吧，它之所以在EM里能简化成那样，其实是因为边缘似然函数本来就是稀疏的，容易看出Q函数那个gap前在一个特定$\mathbf{z}$下很多项其实没有，
所以从指数可以下降到多项式级（从一个笛卡尔积式求和变成了有限累次求和）。真正困难的是边缘似然函数本身是个和的积式的形式，
这就使得直接作用$\log$去把它变成和式变成不可能，但我们注意到EM算法，实际是通过优化边缘似然函数的下界去又获得了可以作用$\log$来简化的优势。

于是我们可以在以下假设下讨论EM算法的用处，

假如欲优化之边缘似然函数内部可以分解成一些乘积

$$
p(\mathbf{X} \mid \theta) =
\sum_{\mathbf{Z}} p(\mathbf{X}, \mathbf{Z} \mid \theta) = 
\sum_{\mathbf{Z}} \prod_{i} g(\mathbf{X}(i),\mathbf{Z}(i),\theta(i))
$$

我们发现这个乘积往往带来很接近0的数值，造成误差，这个数值上的困难让我们回想起对数变化，然而我们注意到这里整体作用$\log$是没用的。
不想没有隐变量时面对独立同分布样本可以直接作用了。那怎么办呢？直观上看，我们可以单独对里面作用$\log$

$$
\sum_{\mathbf{Z}} \log p(\mathbf{X}, \mathbf{Z} \mid \theta) = 
\sum_{\mathbf{Z}} \sum_{i} \log g(\mathbf{X}(i),\mathbf{Z}(i),\theta(i))
$$

那么优化这个函数与优化上面那个函数等价吗？这是否是一种优化下界？可以想到，如果

$$
\log p(\mathbf{X} \mid \theta) =
\log \sum_{\mathbf{Z}} p(\mathbf{X}, \mathbf{Z} \mid \theta) \ge 
\sum_{\mathbf{Z}} \log p(\mathbf{X}, \mathbf{Z} \mid \theta) = 
\sum_{\mathbf{Z}} \sum_{i} \log g(\mathbf{X}(i),\mathbf{Z}(i),\theta(i))
$$

成立，我们就可以通过优化下下界来优化边缘似然函数了，就像KL散度的处理一样。这性质是否起码在某些特殊的$p$上成立？

这并不重要，因为EM算法正是一种解决这个问题的方法，

$$
Q(\theta \mid \theta^{(t)}) = \mathrm{E}_{Z \mid X,\theta^{(t)}} [\log L(\theta;X,Z)] =
\sum_{\mathbf{Z}} p(\mathbf{Z} \mid \mathbf{X},\theta^{(t)}) \log p(\mathbf{X},\mathbf{Z} \mid \theta) =
\sum_{\mathbf{Z}} p(\mathbf{Z} \mid \mathbf{X},\theta^{(t)}) \sum_{i} \log g(\mathbf{X}(i),\mathbf{Z}(i),\theta(i))
$$

通过一些推导，我们可以证明EM算法就是通过优化一个下界来优化原函数的（所以不一定能优化到全局最优。）
所以我们就找到上面那个直观方法的一个替代方案。而这只不过是个花式利用$\log$，图个计算的便利而已。

 ### 采样法估计
 
 如果把参数当做随机变量，要求其后验分布，这问题是不是听起来更难了？然而有时，放弃一些信息反而让我们对问题的本质看的更清楚。
 这时我们完全可以利用马尔科夫蒙特卡洛采样法来获得后验分布，然而怎么用它提炼出唯一估计。只需要参数空间里不停漫步即可，burn-in阶段
 时候参数会向更符合数据的方向几乎头也不回地移动，burn-in结束后就在有一定信度的参数组合上游历，产生出后验分布了。
 
 我们注意到PyMC3之类概率编程“语言”上运用MCMC是特别方便的。
 
 ### 变分推断估计
 
变分推断主要就是从一个简单的分布族$G(\theta)$里去近似边缘后验分布$F(\theta \mid D)$，这是个针对分布的计算技巧，
与EM算法挖尖脑袋用上$\log$差不多。